In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
# Helper Functions

# Init Weights

In [6]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [7]:
# Init Bias

In [8]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [9]:
# CONV2D

In [10]:
def conv2d(x, W):
    # x --> [batch, H, W, Channels]
    # W --> [filter H, filter W, Channels In, CVhannels Out]
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME') 

In [11]:
# Pooling

In [12]:
def max_pooling_2by2(x):
    # x --> [batch, h,w,c]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

In [13]:
# CONVOLUTIONAL LAYER
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

In [22]:
# Normal Layer (Fully Connected Layer)
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [15]:
# PLACEHOLDERS
x = tf.placeholder(tf.float32, shape=[None, 784])

In [16]:
Y_true = tf.placeholder(tf.float32, shape=[None,10])

In [17]:
# Layers
x_image = tf.reshape(x, [-1,28,28,1])

In [19]:
convo_1 = convolutional_layer(x_image,shape=[5,5,1,32])
convo_1_pooling = max_pooling_2by2(convo_1)

In [20]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[5,5,32,64])
convo_2_pooling = max_pooling_2by2(convo_2)

In [24]:
convo_2_flat = tf.reshape(convo_2_pooling, [-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [25]:
# Dropout
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [26]:
y_pred = normal_full_layer(full_one_dropout, 10)

In [28]:
# Loss Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_true, logits=y_pred))

In [29]:
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [30]:
init = tf.global_variables_initializer()

In [32]:
steps = 1000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x, batch_y = mnist.train.next_batch(50)
        
        sess.run(train, feed_dict={x:batch_x,Y_true:batch_y, hold_prob:0.5})
        
        if i%100 == 0:
            
            print("On Step: {}".format(i))
            print("ACCURACY: ")
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(Y_true,1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            print(sess.run(acc, feed_dict={x:mnist.test.images, Y_true:mnist.test.labels, hold_prob:1.0}))
            print('\n')

On Step: 0
ACCURACY: 
0.1243


On Step: 100
ACCURACY: 
0.9498


On Step: 200
ACCURACY: 
0.9647


On Step: 300
ACCURACY: 
0.9692


On Step: 400
ACCURACY: 
0.9757


On Step: 500
ACCURACY: 
0.9773


On Step: 600
ACCURACY: 
0.9803


On Step: 700
ACCURACY: 
0.9804


On Step: 800
ACCURACY: 
0.9815


On Step: 900
ACCURACY: 
0.9838


